In [1]:
import os
import numpy as np
import shutil
import time
import json
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision as tv
import nntools as nt
import torch
import torch.utils.data as data

from models import *
from preprocess import *

In [2]:
images_dir = '/datasets/ee285f-public/VQA2017/'
q_dir = '/datasets/ee285f-public/VQA2017/v2_OpenEnded_mscoco_'
ans_dir = '/datasets/ee285f-public/VQA2017/v2_mscoco_'

train_set = MSCOCODataset(images_dir, q_dir, 
                          ans_dir, mode='train', 
                          image_size=(224, 224))

In [3]:
class ExperimentLoader():
    def __init__(self, input_dir, train_set, batch_size, lr=0.01, weight_decay=0.5):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        self.train_set = train_set
        self.inv_q = {v: k for k, v in self.train_set.vocab_q.items()} 
        self.inv_a = {v: k for k, v in self.train_set.vocab_a.items()} 
        
        self.model = SAN(num_classes=1000, batch_size=batch_size, 
                         vocab_size=len(self.train_set.vocab_q), embedding_dim=500,
                         output_vgg=1024, input_attention=1024, output_attention=512).to(self.device)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), 
                                         lr=lr, 
                                         weight_decay=weight_decay)
        self.batch_size = None
        self.history = None
        self.train_loss = None
        self.train_acc = None
        self.indices = None
        
        os.makedirs(input_dir, exist_ok=True)
        self.checkpoint_path = os.path.join(input_dir, 
                                       "checkpoint.pth.tar")
        self.config_path = os.path.join(input_dir, "config.txt")
        
        # Transfer all local arguments/variables into attributes
        locs = {k: v for k, v in locals().items() if k is not 'self'}
        self.__dict__.update(locs)
        
        if os.path.isfile(self.config_path):
            self.load()
    
    def sample(self, idx):
        self.model = self.model.to(self.device)
        self.model.eval()
        
        with torch.no_grad():
            entry = self.train_set[idx]

            i, q, s, a = entry[0].to(self.device), entry[1].to(self.device), \
                         entry[2], entry[3].to(self.device)
            
            #myimshow(i)
            
            question = []
            for index in range(s):
                question.append(self.inv_q[int(entry[1][index])])

            #print("Question:", ' '.join(question))
            
            i = i.unsqueeze(dim=0)
            q = q.unsqueeze(dim=0)
            s = torch.Tensor([s])

            predicted_answer = F.softmax(self.model.forward(i, q.long(), s.long()))
            _, y_pred = torch.max(predicted_answer, 1)
            
            print("Answer:", self.inv_a[int(y_pred)])            
    
    @property
    def epoch(self):
        return len(self.history)

    def setting(self):
        return {'Net': self.model,
                'Train Set': self.train_set,
                'Optimizer': self.optimizer,
                'BatchSize': self.batch_size}
    
    def __repr__(self):
        """Pretty printer showing the setting of the experiment. This is what
        is displayed when doing ``print(experiment)``. This is also what is
        saved in the ``config.txt`` file.
        """
        string = ''
        for key, val in self.setting().items():
            string += '{}({})\n'.format(key, val)
        return string
    
    def state_dict(self):
        """Returns the current state of the experiment."""
        return {'Net': self.model.state_dict(),
                'Optimizer': self.optimizer.state_dict(),
                'History': self.history,
                'TrainLoss' : self.train_loss,
                'TrainAcc' : self.train_acc,
                'Indices' : self.indices}
    
    def load_state_dict(self, checkpoint):
        self.model.load_state_dict(checkpoint['Net'])
        self.optimizer.load_state_dict(checkpoint['Optimizer'])
        self.history = checkpoint['History']
        self.train_loss = checkpoint['TrainLoss']
        self.train_acc = checkpoint['TrainAcc']
        self.indices = checkpoint['Indices']
        
        for state in self.optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.to(self.device)

    def load(self):
        """Loads the experiment from the last checkpoint saved on disk."""
        checkpoint = torch.load(self.checkpoint_path,
                                map_location=self.device)
        self.load_state_dict(checkpoint)
        del checkpoint

In [4]:
exp = ExperimentLoader(train_set=train_set, batch_size=250, input_dir="newexp_batch250")

In [8]:
indices = exp.indices

ind = indices[0]

In [16]:
exp.sample(1214)

Answer: 1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
